In [1]:
pip install ipytest

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 4.2 MB/s 
     |████████████████████████████████| 1.6 MB 36.0 MB/s 
  Attempting uninstall: pluggy
    Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Attempting uninstall: pytest
    Found existing installation: pytest 3.6.4
    Uninstalling pytest-3.6.4:
      Successfully uninstalled pytest-3.6.4


In [7]:
pip install pytest-mock

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import pytest
import ipytest

ipytest.autoconfig(addopts=["-v"])


In [3]:
%%ipytest

@pytest.fixture
def make_customer_record():
    def _make_customer_record(name):
        return {"name": name, "orders": []}

    return _make_customer_record


def test_customer_records(make_customer_record):
    customer_1 = make_customer_record("Lisa")
    customer_2 = make_customer_record("Mike")
    customer_3 = make_customer_record("Meredith")

.                                                                                            [100%]
1 passed in 0.03s


In [28]:
%%ipytest
import os, tempfile
@pytest.fixture
def cleandir():
    with tempfile.TemporaryDirectory() as newpath:
        old_cwd = os.getcwd()
        os.chdir(newpath)
        yield
        os.chdir(old_cwd)

@pytest.mark.usefixtures("cleandir")
class TestDirectoryInit:
    def test_cwd_starts_empty(self):
        assert os.listdir(os.getcwd()) == []
        with open("myfile", "w") as f:
            f.write("hello")

    def test_cwd_again_starts_empty(self):
        assert os.listdir(os.getcwd()) == []

======================================= test session starts ========================================
platform linux -- Python 3.7.15, pytest-7.2.0, pluggy-1.0.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: mock-3.10.0, typeguard-2.7.1
collecting ... collected 2 items

tmp2bdv74yx.py::TestDirectoryInit::test_cwd_starts_empty PASSED                              [ 50%]
tmp2bdv74yx.py::TestDirectoryInit::test_cwd_again_starts_empty PASSED                        [100%]

======================================== 2 passed in 0.04s =========================================


## Работа с Mock, MagicMock, Patch

In [ ]:
from unittest.mock import Mock, MagicMock, patch

#### Пример использования Mock

In [ ]:
class ProductionClass:
    def closer(self, something):
        something.close()
        
real = ProductionClass()
mock = Mock()
real.closer(mock)
mock.close.assert_called_with()

In [ ]:
mock = Mock()
print(type(mock))
print(type(mock.method()))

<class 'unittest.mock.Mock'>
<class 'unittest.mock.Mock'>


### Аттрибуты Mock

In [ ]:
mock.method.call_count

In [ ]:
mock.call_args

None


In [ ]:
mock.call_args_list

[]

In [ ]:
mock.method_calls

[call.getdoc(),
 call.getdoc(),
 call.getdoc(),
 call.getdoc(),
 call.method.getdoc(),
 call.getdoc(),
 call.getdoc(),
 call.getdoc.getdoc(),
 call.method.getdoc(),
 call.method.getdoc()]

In [ ]:
mock.mock_calls

[call.getdoc(),
 call.getdoc(),
 call.getdoc(),
 call.getdoc(),
 call.method.getdoc(),
 call.getdoc(),
 call.getdoc(),
 call.getdoc.getdoc(),
 call.method.getdoc(),
 call.method.getdoc(),
 call.getdoc(),
 call.getdoc(),
 call.getdoc.getdoc(),
 call.method.getdoc()]

#### mock_calls

In [ ]:
mock = MagicMock()
result = mock(1, 2, 3)
mock.first(a=3)

<MagicMock name='mock.first()' id='140468921558736'>

In [ ]:
mock.second()

<MagicMock name='mock.second()' id='140469047802960'>

In [ ]:
int(mock)

1

In [ ]:
result(1)

<MagicMock name='mock()()' id='140468920666576'>

In [ ]:
expected = [call(1, 2, 3), call.first(a=3), call.second(),
call.__int__(), call()(1)]
mock.mock_calls == expected

True

### Asserts for Mock

In [ ]:
mock = Mock()
mock.method(1, 2, 3, test='wow')

<Mock name='mock.method()' id='140469198952208'>

In [ ]:
mock.method.assert_called_with(1, 2, 3, test='wow')


In [ ]:
mock = Mock(return_value=None)
mock(1, 2, arg='thing')
mock('some', 'thing', 'else')

In [ ]:
mock.assert_any_call(1, 2, arg='thing') # проверяет, был ли вызов вообще когда-либо с данными аргументами


AssertionError: ignored

#### assert_has_calls

In [ ]:
from unittest.mock import call
mock = Mock(return_value=None)
mock(1)
mock(2)
mock(3)
mock(4)
calls = [call(2), call(3)]
mock.assert_has_calls(calls)
calls = [call(4), call(2), call(3)]
mock.assert_has_calls(calls, any_order=True)

### reset_mock

In [ ]:
mock = Mock(return_value=None)
mock('hello')
mock.called



False

In [ ]:
mock.reset_mock()
mock.called

False

### MagicMock side_effect magic

In [ ]:
my_dict = {'a': 1, 'b': 2, 'c': 3}
def getitem(name):
     return my_dict[name]

def setitem(name, val):
    my_dict[name] = val
  
mock = MagicMock(spec_set=dict)
mock.__getitem__.side_effect = getitem
mock.__setitem__.side_effect = setitem

In [ ]:
print(mock['a'], mock['c'])
mock['d']

1 3


KeyError: ignored

In [ ]:
mock = Mock()
mock.__getitem__ = Mock(side_effect=getitem)
mock.__setitem__ = Mock(side_effect=setitem)

### Работа с patch

In [22]:
%%ipytest
import math
 
class TestSmth():
    def test_foo(self, mocker):
        with mocker.patch('math.pow') as mock_foo:
            assert isinstance(math.pow, mocker.MagicMock)

math.pow(2,2)
                    



4.0

.                                                                                            [100%]
========================================= warnings summary =========================================
tmpvygm7grd.py::TestSmth::test_foo
  /content/tmpvygm7grd.py:5: PytestMockWarning: Mocks returned by pytest-mock do not need to be used as context managers. The mocker fixture automatically undoes mocking at the end of a test. This warning can be ignored if it was triggered by mocking a context manager. https://pytest-mock.readthedocs.io/en/latest/remarks.html#usage-as-context-manager

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
1 passed, 1 warning in 0.02s


### Домашнее задание

Задание 1:  
Придумать классы эквивалентности для функции, проверяющей, является ли строка **палиндромом**. Написать эти тесты и реализовать саму функцию. У вас должно получиться **как минимум** 4 теста. Попробуйте придерживаться принципа TDD: в начале реализуйте тесты, а затем — искомую функцию.

Задание 2:  
Для классов Designer и Developer из домашнего задания к 7 вебинару реализуйте фикстуры designer и developer соответственно, такие, чтобы они вовращали функцию, принимающую на вход только seniority, и напишите с их использованием как минимум один тест, проверающий грейд в зависимости от переданного seniority. **Для name и awards мы делаем заглушки dummy. Инициализируем их контантами.**

Пример фикстуры для класса с сигнатурой Customer(name, money):
```
@pytest.fixture
def customer():
  def init_customer(money):
    return Customer(name='Bob',money=money)
  return init_customer
```



Здадание 3:  
Написать тест, проверяющий работу метода **make_format** класса **Rate** из домашнего задания к вебинару 7.

Написать тесты, проверяющие работу метода make_format класса Rate из домашнего задания к 7 вебинару. При этом тесты должны покрывать **все возможные** варианты инициализации класса разными значениями переданных аргументов (format='full'/'value', diff=True/False).
Подсказка: количество написанного кода можно сократить с помощью параметризации тестов, если найти закономерность.

> **!!! Важно!** Тест не должен обращаться к внешним ресурсам. Вам необходимо их замокать (запрос к ресурсу с курсами валют)

> Все валюты из источника тесты проверять не нужно, достаточно выбрать одну валюту. 



